<a href="https://colab.research.google.com/github/reckoner-david/CourseProject/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import sklearn

In [4]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/train.jsonl' , lines=True)

If the above cell does not work. You will have to add the train and test files. Google Colab does not save files. 





In [5]:
def split_context(arc, level=0):
    return arc[level]

In [6]:
def label_convert(arc):
    if arc=='SARCASM':
        return 0
    if arc=='NOT_SARCASM':
        return 1
    return -1

In [7]:
df['context_1'] = df['context'].apply(lambda x: split_context(x, 0))
df['context_2'] = df['context'].apply(lambda x: split_context(x, 1))
df['context_join'] = df['context_2'] + df['context_1']

In [8]:
# Converting the label to 0 or 1. 
df['label1'] = df['label'].apply(lambda x: label_convert(x))

In [9]:
train_df = df[['context_join' , 'response', 'label1']]
train_df.columns=['text_a', 'text_b', 'labels']

In [10]:
train_df.head(10)

,text_a,text_b,labels
0,@USER If your child isn't named Barron ... #Be...,@USER @USER @USER I don't get this .. obviousl...,0
1,@USER @USER having to make up excuses of why y...,@USER @USER trying to protest about . Talking ...,0
2,@USER I ’ ll remember to not support you at th...,@USER @USER @USER He makes an insane about of ...,0
3,@USER But not half as stupid as Schiff looks ....,@USER @USER Meanwhile Trump won't even release...,0
4,@USER They already did . Obama said many times...,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,0
5,@USER because these privileged white boys are ...,@USER @USER @USER -> per your tag line : never...,0
6,@USER @USER do you masturbate to these videos ...,@USER @USER he does ! It excites him then he k...,0
7,@USER Dennis please pass on my love and solida...,"Oh look , it's the #racist @USER offering soli...",0
8,@USER @USER @USER Please forward on to the Soc...,@USER @USER @USER As they are the biggest bull...,0
9,"@USER Careful , Bernie ’ s supporters get trig...",@USER @USER @USER responds to facts by tossing...,0


In [11]:
train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 8,
    'fp16': False, 
    'sliding_window': False, 
    'learning_rate': 3e-05,
    'max_seq_length': 256,
    'do_lower_case': True,
    'train_batch_size': 8,
    'evaluate_during_training': False
}

We are training a model which takes a PAIR of sentences, and returns a label. 
The model input is: (text_a, text_b) Output: Label

In [16]:
# use_cuda is false. Enable this for faster training, without GPU it is very slow to train. 
model = ClassificationModel('distilbert', 'distilbert-base-uncased', args=train_args, use_cuda=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
# model.train_model(train_df)

The above cell is for model training, for demo purposes I also provide the trained model on which we can run the predictions so this cell is not needed to run. It requires GPU's to train.

In [12]:
# If you do not want to run the model training, you can run the uploaded and saved model
model = ClassificationModel('distilbert', '/content/drive/MyDrive/Colab Notebooks/model.zip (Unzipped Files)/checkpoint-5000-epoch-8', args=train_args, use_cuda=False)

In [13]:
df_test = pd.read_json('/content/drive/MyDrive/Colab Notebooks/test.jsonl' , lines=True)

In [14]:
df_test['context_1'] = df_test['context'].apply(lambda x: split_context(x, 0))
df_test['context_2'] = df_test['context'].apply(lambda x: split_context(x, 1))
df_test['context_join'] = df_test['context_2'] + df_test['context_1']
test_df = df_test[['context_join' , 'response']]
test_df.columns=['text_a', 'text_b']

In [15]:
# The predict function takes a list of lists as input.
vals = [[a[0], a[1]] for a in zip(test_df.text_a, test_df.text_b)]

In [16]:
arc = model.predict(vals)
df_test['result'] = arc[0]
df_test['label'] = df_test['result'].apply(lambda x: 'SARCASM' if x == 0 else 'NOT_SARCASM')
df_test[['id' , 'label']].to_csv('answer_distil_1.txt' , index=False, header=False)

In [18]:
# These are the predicted values
df_test[['id' , 'response', 'context', 'label']].head(20)

,id,response,context,label
0,twitter_1,"@USER @USER @USER My 3 year old , that just fi...","[Well now that ’ s problematic AF <URL>, @USER...",NOT_SARCASM
1,twitter_2,@USER @USER How many verifiable lies has he to...,[Last week the Fake News said that a section o...,SARCASM
2,twitter_3,@USER @USER @USER Maybe Docs just a scrub of a...,[@USER Let ’ s Aplaud Brett When he deserves i...,SARCASM
3,twitter_4,@USER @USER is just a cover up for the real ha...,[Women generally hate this president . What's ...,SARCASM
4,twitter_5,@USER @USER @USER The irony being that he even...,"[Dear media Remoaners , you excitedly sharing ...",SARCASM
5,twitter_6,@USER @USER Doesn't matter . Those guys weren'...,[Wilt Chamberlain rejects the skyhook twice in...,SARCASM
6,twitter_7,"@USER @USER @USER So , my #kindnesscascade are...",[I want to start something magical . I don ’ t...,NOT_SARCASM
7,twitter_8,@USER @USER @USER They need to be an MSP to be...,[He ’ s finished . If true this is grooming an...,SARCASM
8,twitter_9,@USER @USER @USER In which Constitution is it ...,[Now students can ’ t bring stones in librarie...,SARCASM
9,twitter_10,@USER @USER ... he says while the GOP is overw...,[One of these things is not like the others . ...,SARCASM
